## Test Data Loading and Overview

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import re
warnings.filterwarnings("ignore")

train_data = pd.read_csv('../data/raw/train.csv').copy()

test_data = pd.read_csv('../data/raw/test.csv').copy()
test_data.drop(columns = 'id', inplace = True)

# Test data shape
print("Test data shape:")
print(test_data.shape)
print("\n" + "-"*50 + "\n")

# Test data description
print("Test data description:")
print(test_data.describe())
print("\n" + "-"*50 + "\n")

# Test data nunique
print("Test data nunique:")
print(test_data.nunique())
print("\n" + "-"*50 + "\n")

# Test data info
print("Test data info:")
test_data.info()
print("\n" + "-"*50 + "\n")

# Test data missing percentage
print("Missing percentage in test data:")
missing_percentage = (test_data.isnull().sum() / len(test_data)) * 100
missing_percentage = missing_percentage.round(2)
print(missing_percentage)
print("\n" + "-"*50 + "\n")

def analyze_columns(test_data):
    for col in test_data.columns:
        print(f"{col} value_counts:\n{test_data[col].value_counts(dropna=False)[:10]}\n")

analyze_columns(test_data)

## Data Visualisering

In [ ]:
# Visualization 1: Distribution of Car Prices (Only for Train Data)
plt.figure(figsize=(10,6))
sns.histplot(train_data['price'], bins=50, kde=True, color='skyblue', label='Train Data', alpha=0.6)
plt.title('Distribution of Car Prices (Train Data)')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Visualization 2: Average Car Price by Model Year (Only for Train Data)
plt.figure(figsize=(10,6))
sns.boxplot(x='model_year', y='price', data=train_data, palette="Set2", fliersize=3)
plt.title('Average Car Price by Model Year (Train Data)')
plt.xlabel('Model Year')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.show()

# Visualization 3: Count of Cars by Model Year (Train vs Test)
plt.figure(figsize=(10,6))
sns.countplot(x='model_year', data=train_data, palette="viridis", alpha=0.6)
sns.countplot(x='model_year', data=test_data, palette="coolwarm", alpha=0.6)
plt.title('Count of Cars by Model Year (Train vs Test)')
plt.xlabel('Model Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(['Train Data', 'Test Data'])
plt.show()

# Visualization 4: Average Price by Car Brand (Only for Train Data)
plt.figure(figsize=(12, 6))
sns.barplot(x='brand', y='price', data=train_data[:10000], color='skyblue', label='Train Data')
plt.title('Average Price by Car Brand (Train Data)')
plt.xlabel('Brand')
plt.ylabel('Average Price')
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Visualization 5: Top 20 Most Common Engine Types (Train vs Test)
top_20_engines_train = train_data['engine'].value_counts().head(20)
top_20_engines_test = test_data['engine'].value_counts().head(20)

plt.figure(figsize=(12,8))
top_20_engines_train.plot(kind='bar', color='skyblue', alpha=0.6, label='Train Data')
top_20_engines_test.plot(kind='bar', color='orange', alpha=0.6, label='Test Data')
plt.title('Top 20 Most Common Engine Types (Train vs Test)')
plt.xlabel('Engine Type')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Visualization 6: Relationship Between Mileage and Price (Only for Train Data)
plt.figure(figsize=(10,6))
sns.scatterplot(x='milage', y='price', data=train_data, color='purple', alpha=0.6, label='Train Data')
plt.title('Relationship Between Mileage and Price (Train Data)')
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.legend()
plt.show()

# Visualization 7: Missing Values Distribution (Train vs Test)
missing_values_train = train_data.isnull().mean() * 100
missing_values_test = test_data.isnull().mean() * 100

missing_values_train = missing_values_train[missing_values_train > 0].sort_values(ascending=False)
missing_values_test = missing_values_test[missing_values_test > 0].sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=missing_values_train.values, y=missing_values_train.index, palette='viridis', label='Train Data', alpha=0.6)
sns.barplot(x=missing_values_test.values, y=missing_values_test.index, palette='coolwarm', label='Test Data', alpha=0.6)
plt.title('Missing Values Distribution (Train vs Test)')
plt.xlabel('Percentage of Missing Values')
plt.ylabel('Features')
plt.tight_layout()
plt.legend()
plt.show()

## Handling NaN values

In [ ]:
# Filling missing values in accident column
test_data['accident'].fillna('None reported', inplace=True)
test_data['accident_mapped'] = test_data['accident'].map({'None reported': 1, 'At least 1 accident or damage reported': 2, 0: 0})

# Filling missing values in clean_title
test_data['clean_title'] = test_data['clean_title'].fillna(0).map({'Yes': 1, 0: 0})

# Filling missing values in int_col
test_data['int_col'] = test_data['int_col'].replace('–', np.nan)
test_data['int_col'].fillna('Black', inplace=True)

# Checking the results
print(test_data.isnull().sum())

## Feature Engineering

In [ ]:
# Brand Column Processing

# Concatenate the unique values of 'brand' from test_data and get the unique brands
unique_brands = test_data['brand'].unique()

# Create a mapping of brands to integer values starting from 1
brand_mapping = {brand: i for i, brand in enumerate(unique_brands, 1)}

# Map the 'brand' column in test_data to the corresponding integer values using the brand_mapping
test_data['brand_mapped'] = test_data['brand'].map(brand_mapping)


# Model Column Processing

# Concatenate the unique values of 'model' from test_data and get the unique models
unique_models = test_data['model'].unique()

# Create a mapping of models to integer values starting from 1
model_mapping = {model: i for i, model in enumerate(unique_models, 1)}

# Map the 'model' column in test_data to the corresponding integer values using the model_mapping
test_data['model_mapped'] = test_data['model'].map(model_mapping)


# Int_col Column Processing

# Concatenate the unique values of 'int_col' from test_data and get the unique interior colors
unique_int_cols = test_data['int_col'].unique()

# Create a mapping of interior colors to integer values starting from 1
int_col_mapping = {int_col: i for i, int_col in enumerate(unique_int_cols, 1)}

# Map the 'int_col' column in test_data to the corresponding integer values using the int_col_mapping
test_data['int_col_mapped'] = test_data['int_col'].map(int_col_mapping)


# Ext_col Column Processing 

# Concatenate the unique values of 'ext_col' from test_data and get the unique exterior colors
unique_ext_cols = test_data['ext_col'].unique()

# Create a mapping of exterior colors to integer values starting from 1
ext_col_mapping = {ext_col: i for i, ext_col in enumerate(unique_ext_cols, 1)}

# Map the 'ext_col' column in test_data to the corresponding integer values using the ext_col_mapping
test_data['ext_col_mapped'] = test_data['ext_col'].map(ext_col_mapping)

# Engine Column Processing

# Concatenate the unique values of 'engine' from test_data and get the unique engine types
unique_engines = test_data['engine'].unique()

# Create a mapping of engine types to integer values starting from 1
engine_mapping = {engine: i for i, engine in enumerate(unique_engines, 1)}

# Map the 'engine' column in test_data to the corresponding integer values using the engine_mapping
test_data['engine_mapped'] = test_data['engine'].map(engine_mapping)

# Fuel Type Encoding
fuel_type_mapping = {
    'Gasoline': 0,
    'Hybrid': 1,
    'E85 Flex Fuel': 2,
    'Diesel': 3,
    'Plug-In Hybrid': 4
}
test_data['fuel_type_mapped'] = test_data['fuel_type'].map(fuel_type_mapping)

In [ ]:
test_data.info()

In [ ]:
current_year = 2024  # Current year for age calculation

luxury_brands = {
    "Ultra Luxury": ["Rolls-Royce", "Lamborghini", "Ferrari", "Bentley", "Aston Martin", "McLaren"],
    "Upper Luxury": ["Porsche", "Maserati", "Land Rover", "Mercedes-Benz", "BMW", "Audi"],
    "Entry-Level Luxury": ["Jaguar", "Lexus", "Genesis", "Cadillac", "Volvo", "Alfa Romeo", "INFINITI", "Acura", "Lincoln", "Rivian", "Hummer", "Lucid", "Karma", "Lotus"]
}

# Calculate car's age based on the model year
test_data["age"] = current_year - test_data["model_year"]

# Extract engine details
def extract_engine_details(df, engine_column):
    # Extract Horsepower (HP) from the engine column
    df['hp'] = df[engine_column].str.extract(r'(\d+\.\d+)HP')[0]
    df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
    df['hp'] = df['hp'].fillna(df['hp'].mean())  # Fill missing values with the mean

    # Extract Engine Size (L) from the engine column
    df['engine_size'] = df[engine_column].str.extract(r'(\d+\.\d+)L')[0]
    df['engine_size'] = pd.to_numeric(df['engine_size'], errors='coerce')

    # Extract Cylinder count (V6, V8, I4, etc.) from the engine column
    df['cylinders'] = df[engine_column].str.extract(r'(\d+)\s?Cylinder')[0]
    df['cylinders'] = pd.to_numeric(df['cylinders'], errors='coerce')

    return df

test_data = extract_engine_details(test_data, 'engine')

# Extract transmission features
def extract_transmission_features(df):
    # Transmission type classification (qual_mapping)
    qual_mapping = {
        'Automatic': 1,
        'Manual': 2,
        'CVT': 3,
        'Dual Shift': 4,
        'Auto-Shift': 5
    }

    # Identify whether the transmission is automatic or manual
    df['is_automatic'] = df['transmission'].str.contains('A/T|Automatic|CVT', case=False, na=False).astype(int)
    df['is_manual'] = df['transmission'].str.contains('M/T|Manual', case=False, na=False).astype(int)

    # Extract the number of transmission speeds
    df['speed'] = df['transmission'].str.extract(r'(\d+)-Speed')[0]
    df['speed'] = pd.to_numeric(df['speed'], errors='coerce')

    # Identify additional features for dual shift and auto shift
    df['has_dual_shift'] = df['transmission'].str.contains('Dual Shift', case=False, na=False).astype(int)
    df['has_auto_shift'] = df['transmission'].str.contains('Auto-Shift', case=False, na=False).astype(int)

    # Map transmission type to numeric values
    df['transmission_type'] = df['transmission'].apply(lambda x: qual_mapping.get(x, -1))  # -1 for unknown types

    return df

test_data = extract_transmission_features(test_data)

# Create other features like rare fuel types and accident history
def create_other_features(df):
    # Create a binary feature for rare fuel types (other than Petrol or Diesel)
    df['rare_fuel_type'] = df['fuel_type'].apply(lambda x: 0 if x in ['Petrol', 'Diesel'] else 1)

    # Identify if the transmission is automatic
    df['is_automatic'] = df['transmission'].apply(lambda x: 1 if x == 'Automatic' else 0)

    # Identify if the car has an accident history
    df['has_accident_history'] = df['accident'].apply(lambda x: 1 if x != 'None reported' else 0)

    return df

test_data = create_other_features(test_data)

# Classify brands into luxury categories
def classify_brand(brand):
    for category, brands in luxury_brands.items():
        if brand in brands:
            return category
    return 'Non-Luxury'

test_data['luxury_category'] = test_data['brand'].apply(classify_brand)

# Map luxury categories to numeric values
test_data['luxury_category'] = test_data['luxury_category'].map({
    'Non-Luxury': 0,
    'Upper Luxury': 1,
    'Entry-Level Luxury': 2,
    'Ultra Luxury': 3
})

In [ ]:
print(test_data.info())
print('\n')
print(test_data.isnull().sum())

In [ ]:
def fill_missing_values(df):
    # Fill missing values in numerical columns with the mean
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numerical_columns:
        df[col] = df[col].fillna(df[col].mean())

    # Fill missing values in categorical columns with the mode (most frequent value)
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])

    return df

# Apply the missing value filling function to the data
test_data = fill_missing_values(test_data)

In [ ]:
print(test_data.info())
print('\n')
print(test_data.isnull().sum())

In [ ]:
test_data.head()

In [ ]:
test_data.to_csv('../data/processed/test_data_processed.csv')